# CNN backbone + NN regresion model

Libraries

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.nn import functional as F
import timm
from tqdm import tqdm

Parameters

In [18]:
# Path control
path = '../input/petfinder-pawpularity-score/'
img_dir = path + 'train/'
label_path = path + 'train.csv'

# Parameters
random_seed = 17
val_size = 0.2
img_size = 224

#Hiperparameters
NUM_EPOCHS = 3
BATCH_SIZE = 32
LEARNING_RATE = 0.00001
MOMENTUM = 0.9
F_OPTIMIZER = optim.SGD
LOSS_FUNCTION = nn.MSELoss()
MODEL_NAME='tf_efficientnet_b0_ns'

# To make transformations of the images
transform = transforms.Compose(
                   [
                    transforms.ToTensor(),
                    transforms.Resize((img_size,img_size)),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])

Setting randomness

In [3]:
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=random_seed)

Device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


Define pytorch dataset object

In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, label_dataframe, img_name, label_name, meta_x, transform=None):
        self.img_dir = img_dir
        self.img_label = label_dataframe
        self.transform = transform
        self.img_name = img_name
        self.label_name = label_name
        self.meta_x = meta_x
        
    def __len__(self):
        return len(self.img_label)

    def __getitem__(self, ix):
        meta_x_ix = self.img_label[self.meta_x].iloc[ix]
        meta_x_ix = torch.tensor(meta_x_ix, dtype=torch.float32)
        img_path = self.img_dir + self.img_label[self.img_name][ix] + '.jpg'
        X_ix = plt.imread(img_path)
        Y_ix = self.img_label[self.label_name][ix]
        Y_ix = torch.tensor(Y_ix, dtype=torch.float32)
        if self.transform:
            X_ix = self.transform(X_ix)
        return (X_ix, meta_x_ix) , Y_ix

Instances Dataset & Dataloader objects

In [6]:
df = pd.read_csv(label_path)
df['nPawpularity']=(df['Pawpularity'] - df['Pawpularity'].min()) / (df['Pawpularity'].max() - df['Pawpularity'].min())
df_train, df_val = train_test_split(df, test_size=0.2, random_state=random_seed)
df_train.reset_index(inplace=True)
df_val.reset_index(inplace=True)
df

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,nPawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,0.626263
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,0.414141
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,0.272727
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,0.141414
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,0.717172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9907,ffbfa0383c34dc513c95560d6e1fdb57,0,0,0,1,0,0,0,0,0,0,0,1,15,0.141414
9908,ffcc8532d76436fc79e50eb2e5238e45,0,1,1,1,0,0,0,0,0,0,0,0,70,0.696970
9909,ffdf2e8673a1da6fb80342fa3b119a20,0,1,1,1,0,0,0,0,1,1,0,0,20,0.191919
9910,fff19e2ce11718548fa1c5d039a5192a,0,1,1,1,0,0,0,0,1,0,0,0,20,0.191919


In [7]:
meta_x_columns = [
    'Subject Focus','Eyes','Face','Near','Action','Accessory', 
    'Group', 'Collage','Human','Occlusion','Info','Blur'
    ]

dataset = {
    'train': Dataset(img_dir, df_train, 'Id', 'Pawpularity', meta_x_columns, transform=transform),
    'val'  : Dataset(img_dir, df_val  , 'Id', 'Pawpularity', meta_x_columns, transform=transform)
}

dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True),
    'val'  : torch.utils.data.DataLoader(dataset['val']  , batch_size=BATCH_SIZE, shuffle=False)
}

Tailored network based on efficientnet

In [8]:
#device = 'cpu'

In [24]:
class Tailored_Net(nn.Module):
    def __init__(
        self, timm_model_name, len_meta_x, target_size, dropout_rate=0, pretrained=True, freezepretrained=False):
        super().__init__()
        self.efficientmodel = timm.create_model(timm_model_name, pretrained=pretrained)
        self.n_features = self.efficientmodel.classifier.in_features
        self.efficientmodel.classifier = nn.Identity()
        self.batchnorm = nn.BatchNorm1d(num_features=self.n_features)
        self.fc1 = nn.Linear(self.n_features + len_meta_x, 64)
        self.fc2 = nn.Linear(64, target_size)
        self.dropout = nn.Dropout(dropout_rate)
        # freeze weights of efficientnet
        if freezepretrained:
            for param in self.efficientmodel.parameters():
                param.requires_grad = False

    def forward(self, image, meta_x):
        # efficient net
        x = self.efficientmodel(image)
        # flatten x
        x = x.view(x.size(0), -1)
        # batchnorm
        x = self.batchnorm(x)
        # concat features
        x = torch.cat( (x, meta_x) , dim=1 )
        # fully connected layer + ReLu + Dropout
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        # fully connected layer + ReLu
        x = self.fc2(x)
        x = F.relu(x)
        # drop dimension of size 1 (same as .view(-1)) (if not, broadcasting problems with y ground truth)
        x = x.squeeze()
        return x

In [25]:
net = Tailored_Net(timm_model_name=MODEL_NAME, len_meta_x=12, target_size=1, dropout_rate=0.3, pretrained=True, freezepretrained=True)
net.to(device) 

Tailored_Net(
  (efficientmodel): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, 

Fake and Real Batch examples to net inference

In [26]:
tensordeprueba = torch.rand(5, 3, 224, 224).to(device)
metadatosdeprueba = torch.rand(5, 12).to(device)
net.eval()
y_hat = net(tensordeprueba, metadatosdeprueba)
y_hat

tensor([0., 0., 0., 0., 0.], device='cuda:0', grad_fn=<SqueezeBackward0>)

In [27]:
# Get 1 batch from dataloader['train']
(X_batch, meta_x_batch), y = next(iter(dataloader['train']))
X_batch = X_batch.to(device)
meta_x_batch = meta_x_batch.to(device)
net.eval()
net(X_batch, meta_x_batch)

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0119, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0497, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<SqueezeBackward0>)

Train the model

In [13]:
#device = 'cpu'

In [28]:
def fit(model, dataloader, epochs=NUM_EPOCHS):
    criterion = LOSS_FUNCTION
    optimizer = F_OPTIMIZER(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            # get the inputs; data is a list of [inputs, labels]
            (X, X_meta), y = batch
            X, X_meta, y = X.to(device), X_meta.to(device), y.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            y_hat = model(X, X_meta)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()

            # save loss
            train_loss.append(loss.item())
            bar.set_description(f"loss {np.mean(train_loss):.5f} rmse {np.mean(train_loss)**0.5:.5f}")

        bar = tqdm(dataloader['val'])
        val_loss = []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                # get the inputs; data is a list of [inputs, labels]
                (X, X_meta), y = batch
                X, X_meta, y = X.to(device), X_meta.to(device), y.to(device)

                # forward
                y_hat = model(X, X_meta)
                loss = criterion(y_hat, y)

                # save loss
                val_loss.append(loss.item())
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_rmse {np.mean(val_loss)**0.5:.5f}")

        print(f"::::Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} rmse {np.mean(train_loss)**0.5:.5f} val_rmse {np.mean(val_loss)**0.5:.5f}::::")


In [29]:
fit(net, dataloader, epochs=NUM_EPOCHS)

  0%|          | 0/248 [00:00<?, ?it/s]

::::Epoch 1/3 loss 1173.58132 val_loss 477.26034 rmse 34.25757 val_rmse 21.84629::::


  0%|          | 0/248 [00:00<?, ?it/s]

::::Epoch 2/3 loss 452.58646 val_loss 394.48758 rmse 21.27408 val_rmse 19.86171::::


val_loss 375.99314 val_rmse 19.39054: 100%|██████████| 62/62 [00:41<00:00,  1.48it/s]

::::Epoch 3/3 loss 404.21721 val_loss 375.99314 rmse 20.10515 val_rmse 19.39054::::


In [16]:
#fit(net, dataloader, epochs=1)

Save the model

In [17]:
torch.save(net.state_dict(), '../models/effnetb0meta_bs32_lr00001_m09.pt')